# B-tree Implementation for Key-value Store
#### An example project illustring B-tree implementation in a key-value store, with numeric or non-numeric unique keys, range look-up, and functional and performance tests

In [1]:
# To structure code automatically
%load_ext nb_black

<IPython.core.display.Javascript object>

### Node Implementation

In [2]:
import bisect

class Node:
    
    def __init__(self, keys=None, values=None, children=None, parent=None):
        self.keys = keys or []
        self.values = values or []
        self.parent = parent
        self.set_children(children)

    def __len__(self):
        return len(self.values)

    def set_children(self, children):
        self.children = children or []
        for child in self.children:
            child.parent = self

    def is_leaf(self):
        return len(self.children) == 0

    def contains_key(self, key):
        return key in self.keys

    def get_value(self, key):
        for i, k in enumerate(self.keys):
            if k == key:
                return self.values[i]
        print(f'Key Error: "{key}" not found')
        return None

    def _get_insert_index(self, key):
        return bisect.bisect(self.keys, key)

    def _update_entry(self, key, value):
        for i, k in enumerate(self.keys):
            if k == key:
                self.values[i] = value

    def insert_entry(self, key, value):
        if self.contains_key(key):
            self._update_entry(key, value)
        else:
            insert_index = self._get_insert_index(key)
            self.keys.insert(insert_index, key)
            self.values.insert(insert_index, value)
            return insert_index  # insert_index used for insert_child call on split_with_parent

    def split(self):
        if self.parent is None:
            return self._split_no_parent()
        return self._split_with_parent()

    def _split_no_parent(self):
        split_index = len(self) // 2
        key_to_move_up = self.keys[split_index]
        value_to_move_up = self.values[split_index]
        right_node = Node(
            self.keys[split_index + 1 :],
            self.values[split_index + 1 :],
            self.children[split_index + 1 :],
        )
        self.keys = self.keys[:split_index]
        self.values = self.values[:split_index]
        self.children = self.children[: split_index + 1]
        parent = Node([key_to_move_up], [value_to_move_up], [self, right_node])
        return parent

    def _insert_child(self, insert_index, child):
        self.children.insert(insert_index, child)
        child.parent = self

    def _split_with_parent(self):
        split_index = len(self) // 2
        key_to_move_up = self.keys[split_index]
        value_to_move_up = self.values[split_index]
        right_node = Node(
            self.keys[split_index + 1 :],
            self.values[split_index + 1 :],
            self.children[split_index + 1 :],
        )
        self.keys = self.keys[:split_index]
        self.values = self.values[:split_index]
        self.children = self.children[: split_index + 1]
        insert_index = self.parent.insert_entry(key_to_move_up, value_to_move_up)
        self.parent._insert_child(insert_index + 1, right_node)

<IPython.core.display.Javascript object>

### B-tree Implementation

In [107]:
class BTree:
    def __init__(self, threshold_size=2):
        self.root = Node()
        self.height = 0
        self.size = 0
        self.threshold_size = threshold_size
        self.min_value = min_value
        self.max_value = max_value

    def _add(self, current_node, key, value):
        if self.height == 0:
            current_node = self.root
        if current_node.is_leaf():
            len_before = len(current_node)
            current_node.insert_entry(key, value)
            len_after = len(current_node)
            if len_after > len_before:
                self.size += 1
            if len_after > self.threshold_size:
                if current_node.parent is None:
                    self.height += 1
                current_node.split()
        else:
            self._add_recursive(self.root, key, value)

    def _add_recursive(self, current_node, key, value):
        child_index = current_node._get_insert_index(key)
        self._add(current_node.children[child_index], key, value)

    def add(self, key, value):
        self._add(self.root, key, value)

    def _range_query(
        self, current_node, range_start, range_end, min_key, max_key,
    ):
        if range_end < min_key or range_start > max_key:
            return []
        results = []
        for i, key in enumerate(current_node.keys):
            if range_start <= key <= range_end:
                results.append(current_node.values[i])
        if not current_node.is_leaf():
            for i, child in enumerate(current_node.children):
                new_min_key = current_node.keys[i - 1] if i > 0 else min_key
                new_max_key = current_node.keys[i] if i < len(current_node) else max_key
                results += self._range_query(
                    child, range_start, range_end, new_min_key, new_max_key
                )
        return results

    def range_query(self, range_start, range_end):
        self._range_query(
            self.root, range_start, range_end, float("-inf"), float("inf")
        )

<IPython.core.display.Javascript object>

In [83]:
bt = BTree()

<IPython.core.display.Javascript object>

In [84]:
bt.add("k", 9)

<IPython.core.display.Javascript object>

In [85]:
bt.root

<IPython.core.display.Javascript object>

In [86]:
bt.root.keys

['k']

<IPython.core.display.Javascript object>

In [87]:
bt.root.values

[9]

<IPython.core.display.Javascript object>

In [88]:
bt.add("b", 4)

<IPython.core.display.Javascript object>

In [89]:
bt.root.keys

['b', 'k']

<IPython.core.display.Javascript object>

In [90]:
bt.root.values

[4, 9]

<IPython.core.display.Javascript object>

In [91]:
bt.add("j", 2)

<IPython.core.display.Javascript object>

In [92]:
bt.root.keys

['b']

<IPython.core.display.Javascript object>

In [93]:
bt.root.values

[4]

<IPython.core.display.Javascript object>

In [94]:
bt.add("j", 5)

<IPython.core.display.Javascript object>

In [95]:
bt.size

4

<IPython.core.display.Javascript object>

In [96]:
bt.height

1

<IPython.core.display.Javascript object>

In [100]:
test = [1, 2, 3, 4, 5]
test[1:3]

[2, 3]

<IPython.core.display.Javascript object>

In [101]:
test[[1:3]]

SyntaxError: invalid syntax (3480118680.py, line 1)

ERROR:root:Cannot parse: 1:7: test[[1:3]]
Traceback (most recent call last):
  File "C:\Users\Teres\anaconda3\lib\site-packages\lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "C:\Users\Teres\anaconda3\lib\site-packages\lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "C:\Users\Teres\anaconda3\lib\site-packages\black.py", line 725, in format_str
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "C:\Users\Teres\anaconda3\lib\site-packages\black.py", line 836, in lib2to3_parse
    raise exc from None
black.InvalidInput: Cannot parse: 1:7: test[[1:3]]
